### 1. Install necessary libraries:

Make sure you have the Google Generative AI LangChain integration installed.

In [ ]:
%pip install --upgrade langchain langchain-experimental langchain-google-genai python-dotenv pyvis google-generativeai google-genai

### 2. Set up your Gemini API Key:

You'll need a Google AI Studio API key.

In [34]:
from dotenv import load_dotenv
import os

# Load the .env file (make sure GOOGLE_API_KEY is present in your .env file)
load_dotenv()

# Get API key from environment variable
# If you don't want to use .env, you can directly assign your API key:
# google_api_key = "YOUR_GEMINI_API_KEY"
google_api_key = os.getenv("GOOGLE_API_KEY")

if not google_api_key:
    raise ValueError("GOOGLE_API_KEY environment variable not set. Please set it in your .env file or directly in the code.")


### 3. The LLM instantiation fro Graph Transformer:

You can specify the Gemini model you want to use (e.g., "gemini-2.5-pro", "gemini-1.5-pro", "gemini-1.5-flash"). "gemini-pro" is a good general-purpose model.

In [35]:
import google.generativeai as genai
import os
from dotenv import load_dotenv

load_dotenv()
google_api_key = os.getenv("GOOGLE_API_KEY")
genai.configure(api_key=google_api_key)

print("List of models that support generateContent:")
for m in genai.list_models():
    if 'generateContent' in m.supported_generation_methods:
        print(m.name)

List of models that support generateContent:
models/gemini-1.0-pro-vision-latest
models/gemini-pro-vision
models/gemini-1.5-pro-latest
models/gemini-1.5-pro-002
models/gemini-1.5-pro
models/gemini-1.5-flash-latest
models/gemini-1.5-flash
models/gemini-1.5-flash-002
models/gemini-1.5-flash-8b
models/gemini-1.5-flash-8b-001
models/gemini-1.5-flash-8b-latest
models/gemini-2.5-pro-preview-03-25
models/gemini-2.5-flash-preview-04-17
models/gemini-2.5-flash-preview-05-20
models/gemini-2.5-flash
models/gemini-2.5-flash-preview-04-17-thinking
models/gemini-2.5-flash-lite-preview-06-17
models/gemini-2.5-pro-preview-05-06
models/gemini-2.5-pro-preview-06-05
models/gemini-2.5-pro
models/gemini-2.0-flash-exp
models/gemini-2.0-flash
models/gemini-2.0-flash-001
models/gemini-2.0-flash-exp-image-generation
models/gemini-2.0-flash-lite-001
models/gemini-2.0-flash-lite
models/gemini-2.0-flash-preview-image-generation
models/gemini-2.0-flash-lite-preview-02-05
models/gemini-2.0-flash-lite-preview
models

In [36]:
from langchain_experimental.graph_transformers import LLMGraphTransformer
from langchain_core.documents import Document
from langchain_google_genai import ChatGoogleGenerativeAI

# Initialize Gemini LLM
# You can choose different models like "gemini-1.5-pro" or "gemini-1.5-flash"
llm = ChatGoogleGenerativeAI(model="gemini-2.5-pro", temperature=0, google_api_key=google_api_key)

# Initialize the LLMGraphTransformer with the Gemini LLM
graph_transformer = LLMGraphTransformer(llm=llm)

### 4. Extract graph data

In [42]:
text = """
Mr. Bean is a British sitcom created by Rowan Atkinson and Richard Curtis, produced by Tiger Aspect Productions and starring Atkinson as the title character. The sitcom consists of 15 episodes that were co-written by Atkinson alongside Curtis and Robin Driscoll; the pilot episode was co-written by Ben Elton. The series originally aired on ITV, beginning with the pilot episode on 1 January 1990[1] and ending with "The Best Bits of Mr. Bean" on 15 December 1995.

Based on a character developed by Atkinson while he was studying for his master's degree at the University of Oxford, the series centres on Mr. Bean, described by Atkinson as "a child in a grown man's body", as he solves various problems presented by everyday tasks and often causes disruption in the process.[2] The series has been influenced by physical comedy actors such as Jacques Tati and those from early silent films.[2]

During its original five-year run, Mr. Bean was met with widespread acclaim and attracted large television audiences. The series was viewed by 18.74 million viewers for the episode "The Trouble with Mr. Bean"[3] and has received a number of international awards, including the Rose d'Or. The series has since been sold in 245 territories worldwide. It has inspired an animated spin-off and two theatrical feature-length films along with Atkinson reprising his role as Mr. Bean for a performance at the London 2012 Summer Olympics opening ceremony, television commercials and several sketches for Comic Relief. The programme carries strong appeal in hundreds of territories worldwide because, in addition to the acclaim from its original run, it uses very little intelligible dialogue, making it accessible to people who know little or no English.
"""

In [43]:
documents = [Document(page_content=text)]
graph_documents = await graph_transformer.aconvert_to_graph_documents(documents)

In [44]:
print(f"Nodes:{graph_documents[0].nodes}")
print(f"Relationships:{graph_documents[0].relationships}")

Nodes:[Node(id='Mr. Bean', type='Sitcom', properties={}), Node(id='Rowan Atkinson', type='Person', properties={}), Node(id='Richard Curtis', type='Person', properties={}), Node(id='Tiger Aspect Productions', type='Company', properties={}), Node(id='Robin Driscoll', type='Person', properties={}), Node(id='Ben Elton', type='Person', properties={}), Node(id='Itv', type='Company', properties={}), Node(id='University Of Oxford', type='University', properties={}), Node(id='Jacques Tati', type='Person', properties={}), Node(id="Rose D'Or", type='Award', properties={}), Node(id='London 2012 Summer Olympics', type='Event', properties={}), Node(id='Comic Relief', type='Organization', properties={})]
Relationships:[Relationship(source=Node(id='Rowan Atkinson', type='Person', properties={}), target=Node(id='Mr. Bean', type='Sitcom', properties={}), type='CREATOR_OF', properties={}), Relationship(source=Node(id='Richard Curtis', type='Person', properties={}), target=Node(id='Mr. Bean', type='Sitcom

### 5. Visualize the graph with Pyvis

In [52]:
from pyvis.network import Network
net = Network(notebook=True, directed=True,
              height="1200px", width="100%", bgcolor="#222222", font_color="white")

# Add nodes
for node in graph_documents[0].nodes:
    net.add_node(node.id, label=node.id, title=node.type, group=node.type) # Customize color for nodes

# Add edges
for relationship in graph_documents[0].relationships:
    net.add_edge(relationship.source.id, relationship.target.id, label=relationship.type.lower())  # Customize color for edges

# Add physics for better layout
net.set_options("""
        {
            "physics": {
                "forceAtlas2Based": {
                    "gravitationalConstant": -100,
                    "centralGravity": 0.01,
                    "springLength": 200,
                    "springConstant": 0.08
                },
                "minVelocity": 0.75,
                "solver": "forceAtlas2Based"
            }
        }
        """)

# Save and display the graph
html_file = "gemini_knowledge_graph.html"
net.show(html_file)
print(f"Graph saved to {os.path.abspath(html_file)}")

gemini_knowledge_graph.html
Graph saved to /Users/helmik/Desktop/workspace/knowledge_graph_llm/gemini_knowledge_graph.html
